In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import flopy

import shutil

# Groundwater Modeling and Python Programming

In this exercise, we will use MODPATH to simulate advective transport with the Freyberg flow model. For this exercise, we will use the existing flow model in the `data/freyberg` directory.

The location of the contamination patch and the nodes that the define bounding cells of the patch are calculated below.

In [ ]:
# patch upper left and lower right
xmin, xmax = 250. * 1, 250. * 3
ymin, ymax = (40 - 14) * 250., (40 - 11) * 250. 
csx, csy = [xmin, xmin, xmax, xmax, xmin], [ymin, ymax, ymax, ymin, ymin]
(xmin, ymax), (xmax, ymin)

In [ ]:
# patch edge node coordinates
xmin_node, xmax_node = xmin + 125, xmax - 125
ymin_node, ymax_node = ymin + 125, ymax - 125
(xmin_node, ymax_node), (xmax_node, ymin_node)

--------------------------

### Define the workspace and model names.

In [ ]:
ws = 'work/ex06b'
load_ws = 'data/freyberg/'
name = 'freyberg'
name_tran = name + '_tran'
exe_name = 'mf2005'

### Clean up the existing MODFLOW-2005 and MT3D files

In [ ]:
shutil.rmtree(ws)

## Load the MODFLOW-2005 Model

Load a MODFLOW-2005 model object using `flopy.modflow.Modflow.load()`.

In [ ]:
fnam = name + '.nam'
mf = flopy.modflow.Modflow.load(fnam, exe_name=exe_name,
                                model_ws='data/freyberg/')

### Change the workspace

In [ ]:
mf.change_model_ws(ws)

In [ ]:
dis = mf.get_package('DIS')

In [ ]:
dis.perlen.array, dis.tsmult.array

In [ ]:
perlen = [1e8]
tsmult = [1.]
dis.perlen = perlen
dis.tsmult = tsmult

In [ ]:
dis.perlen.array, dis.tsmult.array

### Add the LMT package for MT3DMS

In [ ]:
lmt = flopy.modflow.ModflowLmt(mf, unitnumber=130)

### Write the model files

In [ ]:
mf.write_input()

### Run the simulation.

In [ ]:
mf.run_model()

## Create and Run the MT3D model

Lets plot the model grid and the location of the contamination patch.

In [ ]:
mm = flopy.plot.PlotMapView(model=mf, layer=0)

fig = plt.gcf()
fig.set_size_inches(3, 6)

mm.plot_bc('RIV')
mm.plot_bc('WEL')
mm.plot_ibound()

mm.plot_grid(lw=0.5, color='black')

ax = plt.gca()
ax.fill(csx, csy, color='#e534eb')

Find the row column location of the contamination patch using the groundwater model grid object `modelgrid.intersect()` method. The upper left and lower right node coordinates of the cells containing the contamination patch are (375, 7125) and (625, 6625).

In [ ]:
itmin, jtmin = mf.modelgrid.intersect(xmin_node, ymax_node)
itmax, jtmax = mf.modelgrid.intersect(xmax_node, ymin_node) 
(itmin, jtmin), (itmax, jtmax)

In [ ]:
nlay, nrow, ncol = mf.dis.nlay, mf.dis.nrow, mf.dis.ncol
nlay, nrow, ncol

## Create the MT3D files

In [ ]:
# set transport values
ncomp = 1
prsity = 0.01
icbund = np.ones((nlay, nrow, ncol), dtype=np.int)
sconc = np.zeros((nlay, nrow, ncol), dtype=np.float32)

#x_conc = 1000.
#y_conc = 9500.
#col_conc = int(x_conc / 250.) + 1
#row_conc = 40 - int(y_conc / 250.) + 1
#icbund[0, row_conc, col_conc] = -1
#sconc[0, row_conc, col_conc] = 1.
for i in range(itmin, itmax+1):
    for j in range(jtmin, jtmax+1):
        icbund[0, i, j] = -1
        sconc[0, i, j] = 1.
        

al = 10.
trpt = 0.1
trpv = 0.1
dmcoef = 1.e-9
dt0 = 100. #1.e6
ttsmult = 1.05
ttsmax = 86400
nprs = -1  #frequency of saving results

# create the model and add packages
mtexe = 'mt3dms'
tm = flopy.mt3d.Mt3dms(modelname=name_tran,
                         modflowmodel=mf,
                         exe_name=mtexe,
                         model_ws=ws,
                         ftlfilename='mt3d_link.ftl')
btn = flopy.mt3d.Mt3dBtn(tm, ncomp=ncomp, prsity=prsity, icbund=icbund,
                         sconc=sconc, dt0=dt0, ttsmult=ttsmult, ttsmax=ttsmax, nprs=nprs)
adv = flopy.mt3d.Mt3dAdv(tm, mixelm=0)
dsp = flopy.mt3d.Mt3dDsp(tm, al=al, trpt=trpt, trpv=trpv, dmcoef=dmcoef)
ssm = flopy.mt3d.Mt3dSsm(tm, crch={0: 0},
                           stress_period_data={0: [[0, 0, 0, 0, 1]]})
gcg = flopy.mt3d.Mt3dGcg(tm)

### Write the MT3D datasets

In [ ]:
tm.write_input()

### Run MT3D

In [ ]:
tm.run_model(silent=True, normal_msg='Program completed')

## Post-Process the MODFLOW and MT3D Results

### Load MODFLOW and MT3D results from the head and ucn files

Load the MODFLOW heads from the hds file

In [ ]:
hname = name + '.hds'
hname

In [ ]:
fpth = os.path.join(ws, hname)
hobj = flopy.utils.HeadFile(fpth)
hds = hobj.get_data()

Load the concentrations from the UCN file

In [ ]:
cpth = os.path.join(ws, 'MT3D001.UCN')
cobj = flopy.utils.UcnFile(cpth)

In [ ]:
#cobj.get_data(totim=cobj.get_times()[-2])

### Plot the heads and pathlines

In [ ]:
ncols = 2 #len(cobj.get_times())

times = [cobj.get_times()[0], cobj.get_times()[-1]]

figsize = (6, 6)
fig, axes = plt.subplots(nrows=1, ncols=ncols, figsize=figsize)

for idx, totim in enumerate(times):
    ax = axes[idx]
    mm = flopy.plot.PlotMapView(model=mf, ax=ax)
    concs = cobj.get_data(totim=totim)
    #concs[concs < 0] = 0
    mm.contour_array(concs, masked_values=[0], levels=[0.05, 0.1, 0.5, 1.],
                    linewidths=0.75)


    mm.plot_bc('RIV')
    mm.plot_bc('WEL')
    mm.plot_ibound()
    
    ax.fill(csx, csy, color='#e534eb', zorder=100, alpha=.75)

fig.tight_layout()